## Start

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import copy
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import yaml
import tqdm


In [ ]:
all_qs = []
for topic in glob.glob('../questions/topics/*'):
    if 'Demo' not in topic:
        with open(f"{topic}/questions.yaml") as f:
            all_qs.extend(yaml.safe_load(f))

all_qs = pd.DataFrame(all_qs)
correct_answers = dict(zip(all_qs['Id'], all_qs['MultipleChoice'].apply(lambda r: r['Correct'])))

In [ ]:
all_user_data = []

for file in glob.glob('../backend/users/*.yaml'):
    if 'demo' not in file:
        with open(file) as f:
            data = yaml.safe_load(f)

        if 'pretest' in data:
            data['pretest'] = pd.DataFrame(data['pretest'])
        if 'questionSchedule' not in data:
            continue
        
        qSchedule = []
        for i, day in enumerate(data['questionSchedule']):
            for j, q in enumerate(day):
                q['day'] = i
                q['numInDay'] = j
                qSchedule.append(q)
        data['questionSchedule'] = pd.DataFrame(qSchedule)

        if 'posttestA' in data:
            data['posttestA'] = pd.DataFrame(data['posttestA'])
        if 'posttestB' in data:
            data['posttestB'] = pd.DataFrame(data['posttestB'])
        if 'sleepData' in data:
            data['sleepData'] = np.array([float(d['numHours']) for d in data['sleepData']])

        all_user_data.append(data)

all_user_data = pd.DataFrame(all_user_data)

In [ ]:
# list all emails (after the @ sign) from all_user_data["email"]
all_user_data["email"].apply(lambda x: x.split("@")[1]).value_counts()

In [ ]:
len(all_user_data)

In [ ]:
# bad_emails = []
bad_emails = ['madison.evans@som.umaryland.edu', 'puja.patel@som.umaryland.edu', 'kran2@jh.edu', 'charles1@usf.edu']

'''
# shamil34@jhmi.edu 0.3103448275862069
# shannon.hanggodo@quinnipiac.edu 0.26153846153846155
madison.evans@som.umaryland.edu 0.0
# aweitzn1@jh.edu 0.3142857142857143
# mkubica1@jhmi.edu 0.30158730158730157
# mmarani1@jhmi.edu 0.2564102564102564
# aatkin31@jhmi.edu 0.35526315789473684
puja.patel@som.umaryland.edu 0.04225352112676056
# slin86@jhmi.edu 0.328125
# kkuhn14@jh.edu 0.3442622950819672
# kzhu24@jhmi.edu 0.18181818181818182
# xdai9@jhmi.edu 0.3770491803278688
# lyoung1090@gmail.com 0.22857142857142856
charles1@usf.edu 0.20588235294117646
kran2@jh.edu 0.0
'''

finished_study_data = all_user_data[all_user_data['status'].isin(['posttestDone', 'studyDone', 'posttestPartADone']) & ~all_user_data['email'].isin(bad_emails)]
finished_users = finished_study_data['email'].unique()
print(f"Participants who finished study portion: {len(finished_study_data)}\n{finished_users}")

In [ ]:
finished_post_data = all_user_data[all_user_data['status'].isin(['posttestDone']) & ~all_user_data['email'].isin(bad_emails)].reset_index()
print(f"Participants who finished all post-tests: {len(finished_post_data)}\n{finished_post_data['email'].to_list()}")

## Fraction contested

Calculating fraction contested overall and per user

In [ ]:
with open('../backend/scoring/contestedEvaluations.yaml') as f:
    contested_evaluations = pd.DataFrame(yaml.safe_load(f))

In [ ]:
count = 0
users_seen = set()
for _, row in contested_evaluations.iterrows():
    if row['user'] not in finished_users:
      continue
    user_row = finished_post_data[finished_post_data['email'] == row['user']]
    qSched = user_row['questionSchedule'].iloc[0]
    qSched_row = qSched[qSched['qid'] == row['QID']].iloc[0]
    if qSched_row['modality'] == 'mc':
      print('oops..')
      print(row)
      contested_evaluations = contested_evaluations[~((contested_evaluations['user'] == row['user']) & (contested_evaluations['timestamp'] == row['timestamp']))]

    if row['user'] not in users_seen:
      count = count + 1
      users_seen.add(row['user'])

print(f"{count} users contested at least 1 score")
contested_evaluations

In [ ]:
total_questions = 120

frac_contested = len(contested_evaluations[contested_evaluations['user'].isin(finished_users)]) / (len(finished_users) * total_questions)
frac_contested_per_user = {user: (contested_evaluations['user'] == user).sum() / total_questions for user in finished_users}
print(f"Percentage of overall responses contested: {100 * frac_contested:.2f}% +/- {100 * scipy.stats.sem([x for x in frac_contested_per_user.values()]):.2f}%")

plt.title('Percentage of responses contested per user')
plt.bar(frac_contested_per_user.keys(), np.array(list(frac_contested_per_user.values())) * 100)
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
contested_evaluations['correct'] = [correct_answers[qid] for qid in contested_evaluations['QID']]
contested_evaluations

In [ ]:
questions = contested_evaluations['QID'].map(lambda qid: all_qs['Question'][all_qs['Id'] == qid].item())
contested_evaluations_save = pd.DataFrame({
    'QID': contested_evaluations['QID'],
    'Figures': ['']*len(contested_evaluations),
    'Question': questions,
    'User Response': contested_evaluations['userResponse'],
    'Correct Response': contested_evaluations['correct'],
    'Was User Correct?': ['']*len(contested_evaluations),
    'Comments': ['']*len(contested_evaluations),
    'SAIL Score': contested_evaluations['score'],
    'User': contested_evaluations['user']
})

writer = pd.ExcelWriter('contested_evaluations.xlsx', engine='xlsxwriter')
contested_evaluations_save.to_excel(writer, sheet_name='Sheet1')
writer.sheets['Sheet1'].set_default_row(300)

for i, qid in enumerate(contested_evaluations['QID']):
    topic = ' '.join(qid.split(' ')[:-1])
    figs = all_qs['Figures'][all_qs['Id'] == qid].item()
    if isinstance(figs, list):
        for f in figs:
            writer.sheets['Sheet1'].insert_image(i+1, 2, f'../questions/topics/{topic}/{f}')

writer.save()

In [ ]:
import docx

document = docx.Document()
questions = contested_evaluations['QID'].map(lambda qid: all_qs['Question'][all_qs['Id'] == qid].item())

for i in range(len(contested_evaluations)):
    r = document.add_paragraph().add_run()
    r.add_text(f"QID: {contested_evaluations['QID'][i]}")
    r.add_text(f"Question: {questions[i]}")
    r.add_text(f"User Response {contested_evaluations['userResponse'][i]}")
    r.add_text(f"Correct Response: {contested_evaluations['correct'][i]}")

document.save('contested_evaluations.docx')

contested_evaluations_save = pd.DataFrame({
    'QID': contested_evaluations['QID'],
    'Figures': ['']*len(contested_evaluations),
    'Question': questions,
    'User Response': contested_evaluations['userResponse'],
    'Correct Response': contested_evaluations['correct'],
    'Was User Correct?': ['']*len(contested_evaluations),
    'Comments': ['']*len(contested_evaluations),
    'SAIL Score': contested_evaluations['score'],
    'User': contested_evaluations['user']
})

writer = pd.ExcelWriter('contested_evaluations.xlsx', engine='xlsxwriter')
contested_evaluations_save.to_excel(writer, sheet_name='Sheet1')
writer.sheets['Sheet1'].set_default_row(300)

for i, qid in enumerate(contested_evaluations['QID']):
    topic = ' '.join(qid.split(' ')[:-1])
    figs = all_qs['Figures'][all_qs['Id'] == qid].item()
    if isinstance(figs, list):
        for f in figs:
            writer.sheets['Sheet1'].insert_image(i+1, 2, f'../questions/topics/{topic}/{f}')

writer.save()

## Reliability of voice transcription

Calculating fraction of times they edited voice transcription, overall and per user

In [ ]:
num_voice_responses = 60
voice_edited_responses = finished_study_data['questionSchedule'].apply(
    lambda qs: qs[(qs['modality'] == 'voice') & (qs['userResponse'].str.strip().str.lower() != qs['originalResponse'].str.strip().str.lower())])

In [ ]:
frac_edited_per_user = voice_edited_responses.apply(len) / num_voice_responses

print(f"Percentage of overall responses edited: {100 * frac_edited_per_user.mean():.2f}% +/- {100 * frac_edited_per_user.sem():.2f}%")

plt.title('Percentage of responses edited per user')
plt.bar(finished_study_data['email'], frac_edited_per_user * 100)
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Study example response edits
voice_edited_responses[finished_study_data['email'] == 'andrewbharris@jhmi.edu'].squeeze()


## Study Performance per Modality

In [ ]:
def calc_study_accuracy(qs, modality=None):
    if modality is not None:
        qs = qs[qs['modality'] == modality]
    return qs['score'].mean()

overall = finished_study_data['questionSchedule'].apply(lambda qs: calc_study_accuracy(qs)).mean()
voice = finished_study_data['questionSchedule'].apply(lambda qs: calc_study_accuracy(qs, 'voice')).mean()
voiceless = finished_study_data['questionSchedule'].apply(lambda qs: calc_study_accuracy(qs, 'voiceless')).mean()
mc = finished_study_data['questionSchedule'].apply(lambda qs: calc_study_accuracy(qs, 'mc')).mean()

sems = [finished_study_data['questionSchedule'].apply(lambda qs: calc_study_accuracy(qs, modality)).sem() for modality in [None, "voice", "voiceless", "mc"]]

plt.title('Accuracy during study per modality')
plt.bar(['Overall', 'Voice', 'Voiceless', 'MC'], [overall, voice, voiceless, mc], yerr=sems, alpha=1, ecolor='black', capsize=10)
plt.plot()


## Improvement per Modality

In [ ]:
def calc_test_accuracies(test_type, modality, test_method):
    accs = []
    for _, row in finished_post_data.iterrows():
        qSched = row['questionSchedule']
        if modality == 'all':
            modality_qids = qSched['qid'].unique()
        else:
            modality_qids = qSched['qid'][qSched['modality'] == modality].unique()

        test = row[test_type.split(".")[0]]
        if "." in test_type:
            test = test[test_type.split(".")[1]]
        
        test = pd.DataFrame(test)
        if test_method == 'recognition':
            accuracy = np.mean([test['response'][i] == correct_answers[test['QID'][i]]
                                for i in range(len(test)) if test['QID'][i] in modality_qids])
        elif test_method == 'recall':
            accuracy = np.mean([test['automated_recall_score'][i]
                                for i in range(len(test)) if test['QID'][i] in modality_qids])
        
        accs.append(accuracy)
    
    return accs

pretest_acc = calc_test_accuracies('pretest', 'all', test_method="recognition")
posttest_recall_acc = []
posttest_recognition_acc = []

for posttest_iteration in ['first_posttest', 'second_posttest', 'posttest']:
    recall_testname = f'{posttest_iteration}.A'
    recog_testname = f'{posttest_iteration}.B'
    if posttest_iteration == "posttest":
        recall_testname = "posttestA"
        recog_testname = "posttestB"
    
    posttest_recall_acc.append({modality: calc_test_accuracies(recall_testname, modality, test_method='recall')
                                for modality in ['all', 'voice', 'voiceless', 'mc']})
    posttest_recognition_acc.append({modality: calc_test_accuracies(recog_testname, modality, test_method="recognition")
                                     for modality in ['all', 'voice', 'voiceless', 'mc']})



In [ ]:
def calc_test_stats(test_type, modality):
    stats = []
    for _, row in finished_post_data.iterrows():
        qSched = row['questionSchedule']
        if modality == 'all':
            modality_qids = qSched['qid'].unique()
        else:
            modality_qids = qSched['qid'][qSched['modality'] == modality].unique()

        test = row[test_type.split(".")[0]]
        if "." in test_type:
            test = test[test_type.split(".")[1]]
        
        test = pd.DataFrame(test)
        dataaaa = []
        dataaaa = [i for i in range(len(test)) if test['QID'][i] in modality_qids]
        
        stats.append(len(dataaaa))
    
    return stats


for modality in ["all", "voice", "voiceless", "mc"]:
    # calculate how many questions were included for each post-test test
    arr = calc_test_stats("first_posttest.A", modality)
    # print median, 25th percentile, 75th percentile
    print(modality, np.median(arr), np.percentile(arr, 25), np.percentile(arr, 75))


In [ ]:
plt.rcParams.update({'font.size': 10})

bar_width = 0.5
plt.title('Pre-Test Baseline Recognition Scores per User')
plt.bar(finished_post_data['email'], pretest_acc, alpha=1, ecolor='black', capsize=10)
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(16, 4), dpi=300)
bar_width = 0.375
for i, modality in enumerate(['all', 'voice', 'voiceless', 'mc']):
    ax[i].set_title(f'{modality}' if modality != 'all' else "overall")
    ax[i].set_ylim(0, 1.1)
    ax[i].bar(np.arange(len(finished_post_data)) + bar_width,
              posttest_recognition_acc[0][modality], width=bar_width, label='post-test recognition')
    ax[i].bar(np.arange(len(finished_post_data)) + (2 * bar_width),
              posttest_recall_acc[0][modality], width=bar_width, label='post-test recall')

    ax[i].set_xticks(np.arange(len(finished_post_data)) + 1.5 * bar_width,
                     finished_post_data['email'], rotation=45, ha='right')

plt.suptitle("Post-Test #1")
plt.legend(bbox_to_anchor=(1.1, 1.05))
plt.show()

for i, score in enumerate(posttest_recall_acc[0]["voice"]):
    if score < 0.4:
        print(finished_post_data['email'][i], score)


In [ ]:
mean_posttest_recall_acc = [{modality: np.mean(np.subtract(accs, 0)) for modality, accs in posttest_recall_acc[i].items() if modality != "all"} for i in range(3)] 
mean_posttest_recall_sem = [{modality: scipy.stats.sem(np.subtract(accs, 0)) for modality, accs in posttest_recall_acc[i].items() if modality != "all"} for i in range(3)]
mean_posttest_recognition_acc = [{modality: np.mean(np.subtract(accs, 0)) for modality, accs in posttest_recognition_acc[i].items() if modality != "all"} for i in range(3)]
mean_posttest_recognition_sem = [{modality: scipy.stats.sem(np.subtract(accs, 0)) for modality, accs in posttest_recognition_acc[i].items() if modality != "all"} for i in range(3)]

for i in range(3):
    print(f"Test {i + 1}")
    for modality in ["voice", "voiceless", "mc"]:
      print(f"{modality}")
      print(f"Recall {mean_posttest_recall_acc[i][modality]} +- {mean_posttest_recall_sem[i][modality]}")
      print(f"Recog {mean_posttest_recognition_acc[i][modality]} +- {mean_posttest_recognition_sem[i][modality]}")
    
# mean_posttest_recall_acc = [{modality: np.mean(accs) for modality, accs in posttest_recall_acc[i].items() if modality != "all"} for i in range(3)] 
# mean_posttest_recall_sem = [{modality: scipy.stats.sem(accs) for modality, accs in posttest_recall_acc[i].items() if modality != "all"} for i in range(3)]
# mean_posttest_recognition_acc = [{modality: np.mean(accs) for modality, accs in posttest_recognition_acc[i].items() if modality != "all"} for i in range(3)]
# mean_posttest_recognition_sem = [{modality: scipy.stats.sem(accs) for modality, accs in posttest_recognition_acc[i].items() if modality != "all"} for i in range(3)]


fig, ax = plt.subplots(1, 3, figsize=(16, 4), dpi = 300)
for i in range(3):
  ax[i].bar(np.arange(3), mean_posttest_recall_acc[i].values(), yerr=mean_posttest_recall_sem[i].values(), alpha=1, ecolor='black', capsize=10)
  neg_vals = []
  for val in mean_posttest_recall_acc[i].values():
    if val > 0:
      neg_vals.append(0)
    else:
      neg_vals.append(val)
  ax[i].bar(np.arange(3), neg_vals, alpha=1, color='r')
  ax[i].axis(ymin=0, ymax=1)
  ax[i].set_xticks(np.arange(3), mean_posttest_recall_acc[i].keys())
  ax[i].set_title(f"Test {i + 1}")

fig.suptitle('Post-Test Recall Scores per Learning Modality')
fig.show()


fig, ax = plt.subplots(1, 3, figsize=(16, 4), dpi = 300)
for i in range(3):
  ax[i].bar(np.arange(3), mean_posttest_recognition_acc[i].values(), yerr=mean_posttest_recognition_sem[i].values(), alpha=1, ecolor='black', capsize=10)
  neg_vals = []
  for val in mean_posttest_recognition_acc[i].values():
    if val > 0:
      neg_vals.append(0)
    else:
      neg_vals.append(val)
  ax[i].bar(np.arange(3), neg_vals, alpha=1, color='r')
  ax[i].axis(ymin=0, ymax=1)
  ax[i].set_xticks(np.arange(3), mean_posttest_recognition_acc[i].keys())
  ax[i].set_title(f"Test {i + 1}")

fig.suptitle('Post-Test Recognition Scores per Learning Modality')
fig.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16, 4), dpi = 300)
for i in range(3):
  ax[i].violinplot([accs for _, accs in posttest_recall_acc[i].items()])
  ax[i].axis(ymin=-0.1, ymax=1.1)
  ax[i].set_xticks(np.arange(4) + 0.5, posttest_recall_acc[i].keys())
  ax[i].set_title(f"Test {i + 1}")

fig.align_xlabels()
fig.suptitle('Post-Test Recall Scores per Learning Modality')
fig.show()

fig, ax = plt.subplots(1, 3, figsize=(16, 4), dpi = 300)
for i in range(3):
  ax[i].violinplot([accs for _, accs in posttest_recognition_acc[i].items()])
  ax[i].axis(ymin=-0.1, ymax=1.1)
  ax[i].set_xticks(np.arange(4) + 0.5, posttest_recognition_acc[i].keys())
  ax[i].set_title(f"Test {i + 1}")
fig.suptitle('Post-Test Recognition Scores per Learning Modality')
fig.show()


for i in range(3):
  print(f"Recall Test {i + 1}")
  for key, accs in posttest_recall_acc[i].items():
    ks_result = scipy.stats.shapiro(accs)
    print(f"{key} p-value: {ks_result[1]}")

for i in range(3):
  print(f"Recognition Test {i + 1}")
  for key, accs in posttest_recognition_acc[i].items():
    ks_result = scipy.stats.shapiro(accs)
    print(f"{key} p-value: {ks_result[1]}")

## Forgetting Curve

In [ ]:
modalities_dict = {'voice': 'Oral', 'voiceless': 'Typed', 'mc': 'MC'}
modalities = ['voice', 'voiceless', 'mc']

mean_posttest_recall_acc = [{modality: np.mean(accs) for modality, accs in posttest_recall_acc[i].items()} for i in range(3)] 
mean_posttest_recall_sem = [{modality: scipy.stats.sem(accs) for modality, accs in posttest_recall_acc[i].items()} for i in range(3)]
mean_posttest_recognition_acc = [{modality: np.mean(accs) for modality, accs in posttest_recognition_acc[i].items()} for i in range(3)]
mean_posttest_recognition_sem = [{modality: scipy.stats.sem(accs) for modality, accs in posttest_recognition_acc[i].items()} for i in range(3)]

fig, ax = plt.subplots(1, 2, figsize=(14, 4), dpi=600)
colors = ['#1b9e77', '#0295D9', '#7570b3']

for i, modality in enumerate(modalities):
  ax[0].errorbar(['Day 1', 'Day 7', 'Day 60'], 
                 [mean_posttest_recall_acc[i][modality] for i in range(3)],
                 yerr=[mean_posttest_recall_sem[i][modality] for i in range(3)], 
                 fmt='o-', label=modalities_dict[modality], color=colors[i],
                 linewidth=2)
ax[0].axis(ymin=0, ymax=0.55)
ax[0].set_title("Recall")

# ax[0].set_ylabel('Rate', labelpad=10)


for i, modality in enumerate(modalities):
  ax[1].errorbar(['Day 1', 'Day 7', 'Day 60'], 
                 [mean_posttest_recognition_acc[i][modality] for i in range(3)], 
                 yerr=[mean_posttest_recognition_sem[i][modality] for i in range(3)], 
                 fmt='o-', color=colors[i],
                 linewidth=2)
# ax[1].set_xticks(np.arange(3), ['Day 1', 'Day 7', 'Day 60'])
ax[1].axis(ymin=0, ymax=0.8)
ax[1].set_title("Recognition")

plt.rcParams.update({'font.size': 20})

fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=False, ncol=5)

# less padding for the legend
plt.subplots_adjust(bottom=0)

plt.rcParams.update({'font.size': 20})

fig.show()
fig.savefig('forgetting_curve.png', dpi=600, bbox_inches='tight', transparent=False, pad_inches=0.1, facecolor='white')

## Statistical Comparisons

In [ ]:
# https://www.graphpad.com/guides/prism/latest/statistics/
# https://www.marsja.se/repeated-measures-anova-in-python-using-statsmodels/

from statsmodels.stats.anova import AnovaRM
from scipy.stats import wilcoxon, friedmanchisquare, ttest_rel

In [ ]:
# Two way RM ANOVA to test whether modality, time of post-test, or interaction between modality and time of post-test has an effect on test accuracy 
# interaction null hypothesis is that the difference between the modalities is the same for all three post-tests

recall_twoway = [['subject_id', 'score', 'modality', 'test_iter']]
for test_iter in range(3):
    for modality, accs in posttest_recall_acc[test_iter].items():
      if modality == "all" or modality == "voiceless":
         continue
      for id, score in enumerate(accs):
         recall_twoway.append([id, score, modality, test_iter])

recog_twoway = [['subject_id', 'score', 'modality', 'test_iter']]
for test_iter in range(3):
    for modality, accs in posttest_recognition_acc[test_iter].items():
      if modality == "all" or modality == "voiceless":
         continue
      for id, score in enumerate(accs):
         recog_twoway.append([id, score, modality, test_iter])

recog_twoway_df = pd.DataFrame(recog_twoway[1:], columns=recog_twoway[0])
recall_twoway_df = pd.DataFrame(recall_twoway[1:], columns=recall_twoway[0])

anova2way = AnovaRM(recog_twoway_df, 'score', 'subject_id', within=['modality', 'test_iter'])
res_recog = anova2way.fit()
anova2way = AnovaRM(recall_twoway_df, 'score', 'subject_id', within=['modality', 'test_iter'])
res_recall = anova2way.fit()
print("Recall", res_recall)
print("Recognition", res_recog)

In [ ]:
for i in range(0, 3):
  print(f"Test {i + 1}")
  anova = AnovaRM(recog_twoway_df[recog_twoway_df['test_iter'] == i], 'score', 'subject_id', within=['modality'])
  res_recog = anova.fit()

  anova = AnovaRM(recall_twoway_df[recall_twoway_df['test_iter'] == i], 'score', 'subject_id', within=['modality'])
  res_recall = anova.fit()

  print("Recall", res_recall)
  print("Recognition", res_recog)
  print('-----------------')


In [ ]:
# "Testing the durability/robustness of learning"
# Friedman to test whether modality has an effect on accuracy in the final, 2-month test: Test 3
for i in range(0, 3):
  print(f"Test {i + 1}")
  recall_table = []
  for modality, accs in posttest_recall_acc[i].items():
    if modality == "all":
      continue
    recall_table.append(accs)

  recog_table = []
  for modality, accs in posttest_recognition_acc[i].items():
    if modality == "all":
      continue
    recog_table.append(accs)

  print("Recall", friedmanchisquare(recall_table[0], recall_table[1], recall_table[2]))
  print("Recognition", friedmanchisquare(recog_table[0], recog_table[1], recog_table[2]))
  print('')

In [ ]:
# Post-hoc Analysis for Recall

# test_func = ttest_rel
test_func = wilcoxon

# # How to design post-hoc test to explore which two timepoints have a different difference in accuracy between modalities?
# # Wilcoxon to compare (Voice-MC) at time 1 vs (Voice-MC) at time 3 ? Note, these values have a + or - sign.
# print("Wilcoxon to compare (Voice-MC) at time 1 vs (Voice-MC) at time 3")
# stat, p = test_func(np.subtract(posttest_recall_acc[2]['voice'], posttest_recall_acc[2]['mc']), 
#                     np.subtract(posttest_recall_acc[0]['voice'], posttest_recall_acc[0]['mc']))
# print(f"(Voice-MC) at time 3 != at time 1: {p}")

# print("Wilcoxon to compare (Voice-MC) at time 2 vs (Voice-MC) at time 1")
# stat, p = test_func(np.subtract(posttest_recall_acc[1]['voice'], posttest_recall_acc[1]['mc']), 
#                     np.subtract(posttest_recall_acc[0]['voice'], posttest_recall_acc[0]['mc']))
# print(f"(Voice-MC) at time 2 != at time 1: {p}")


print("--------------------")

print("Recall")
stat, p = test_func(posttest_recall_acc[0]['voice'], posttest_recall_acc[0]['mc'])
print(f"Voice != MC (test 1): {p}")

stat, p = test_func(posttest_recall_acc[0]['voice'], posttest_recall_acc[0]['voiceless'])
print(f"Voice != Voiceless (test 1): {p}")

stat, p = test_func(posttest_recall_acc[1]['voice'], posttest_recall_acc[1]['mc'])
print(f"Voice != MC (test 2): {p}")

stat, p = test_func(posttest_recall_acc[1]['voice'], posttest_recall_acc[1]['voiceless'])
print(f"Voice != Voiceless (test 2): {p}")

stat, p = test_func(posttest_recall_acc[2]['voice'], posttest_recall_acc[2]['mc'])
print(f"Voice != MC (test 3): {p}")

stat, p = test_func(posttest_recall_acc[2]['voice'], posttest_recall_acc[2]['voiceless'])
print(f"Voice != Voiceless (test 3): {p}")

print("--------------------")
print("Recognition")

# Post-hoc Analysis for Recognition
stat, p = test_func(posttest_recognition_acc[0]['mc'], posttest_recognition_acc[0]['voice'])
print(f"MC != Voice (test 1): {p}")

stat, p = test_func(posttest_recognition_acc[1]['mc'], posttest_recognition_acc[1]['voice'])
print(f"MC != Voice (test 2): {p}")

stat, p = test_func(posttest_recognition_acc[2]['mc'], posttest_recognition_acc[2]['voice'])
print(f"MC != Voice (test 3): {p}")

## Multivariate Regression

### Participant characteristics

In [ ]:
## Participant characteristics
participantInfo = pd.read_csv('/home/arpan/SAIL/backend/users/participants.csv')

# level of schooling
# birthday
# gender
# first language
# preferred language
# country of origin

# filter participantInfo to only include participants who have completed the study
participantInfo = participantInfo[participantInfo['Username'].isin(finished_post_data['email'])]
participantInfo.reset_index(inplace=True, drop=True)

import datetime
def calculate_age(x):
  bday = x['Birthday']
  try:
    age = (datetime.datetime.now() - datetime.datetime.strptime(bday, "%Y-%m-%d")).days / 365
  except Exception as e:
    print(x['Username'], bday)
    age = 26
  return age

participantInfo['age'] = participantInfo.apply(lambda x: calculate_age(x), axis=1)


# in participantInfo['Level of Schooling'], replaces instances of "M4 at UMSOM" with "Medical Student - Year 4" and "MS2" with "Medical Student - Year 2"
participantInfo['Level of Schooling'] = participantInfo['Level of Schooling'].replace("M4 at UMSOM", "Medical Student - Year 4")
participantInfo['Level of Schooling'] = participantInfo['Level of Schooling'].replace("MS2", "Medical Student - Year 2")


participantInfo

# create a XLSX file that can have multiple sheets
writer = pd.ExcelWriter('participantInfo.xlsx', engine='xlsxwriter')

for col in ["Level of Schooling", "age", "Gender", "In which country were you born?", "What was the first language you learned?", "What is your preferred language?"]:
  if col == "age":
    # create a table of summary stats for participantInfo['age'] and save it to a sheet of the xlsx files
    summary_stats = participantInfo['age'].describe()
    # add SEM to summary stats
    summary_stats['SEM'] = participantInfo['age'].sem()
    summary_stats.to_excel(writer, sheet_name=col)
    continue

  value_counts = participantInfo[col].value_counts(normalize=True)
  value_counts = value_counts.to_frame()
  value_counts['count'] = participantInfo[col].value_counts()
  print(value_counts)
  
  # save value counts to a sheet of the xlsx file
  col = col.replace(" ", "_")
  col = col.replace("?", "")
  col = col[:31]
  value_counts.to_excel(writer, sheet_name=col)

# save the xlsx file
writer.save()

In [ ]:
import datetime


# create pandas df from a csv file
participantInfo = pd.read_csv('/home/arpan/SAIL/backend/users/participants.csv')

# trainingDict = {
#   "Orthopedic Resident - Year 5": 9,
#   "Orthopedic Resident - Year 4": 8,
#   "Orthopedic Resident - Year 3": 7,
#   "Orthopedic Resident - Year 2": 6,
#   "Orthopedic Resident - Year 1": 5,
#   "Medical Student - Year 4": 4,
#   "Medical Student - Year 3": 3,
#   "Medical Student - Year 2": 2,
#   "Medical Student - Year 1": 1,
#   "MS2": 2,
#   "M4 at UMSOM": 4,
#   "M2": 2,
#   "Medical Student - MS3": 3
# }

trainingDict = {
  "Orthopedic Resident - Year 5": 6,
  "Orthopedic Resident - Year 4": 5,
  "Orthopedic Resident - Year 3": 4,
  "Orthopedic Resident - Year 2": 3,
  "Orthopedic Resident - Year 1": 2,
  "Medical Student - Year 4": 1,
  "Medical Student - Year 3": 0.5,
  "Medical Student - Year 2": 0,
  "Medical Student - Year 1": 0,
  "MS2": 0,
  "M4 at UMSOM": 1,
  "M2": 0,
  "Medical Student - MS3": 0.5
}

mat = []
for _, row in finished_post_data.iterrows():
  qSched = row['questionSchedule']
  sleepData = row['sleepData']
  meanSleep = np.mean(sleepData)
  varSleep = np.var(sleepData)
  minSleep = np.min(sleepData)
  maxSleep = np.max(sleepData)
  lastSleep = sleepData[-1]

  email = row['email']
  curr_participant = participantInfo[participantInfo['Username'] == email].iloc[0]
  bday = curr_participant['Birthday']
  
  try:
    age = (datetime.datetime.now() - datetime.datetime.strptime(bday, "%Y-%m-%d")).days / 365
  except Exception as e:
    print(email, bday)
    age = 26
    
  gender = curr_participant['Gender'] # CATEGORICAL !
  trainingLevel = trainingDict[curr_participant['Level of Schooling']]

  recall_scores = []
  recog_scores = []
  # 1st post-test
  for y in ['A', 'B']:
    for x in row['first_posttest'][y]:
      if y == 'A':
        score = x['automated_recall_score']
        recall_scores.append(score)
      else:
        QID = x['QID']
        score = 1 if x['response'] == correct_answers[QID] else 0
        recog_scores.append(score)

  test_time = 1
  mat.append([
    meanSleep,
    varSleep,
    minSleep,
    maxSleep,
    lastSleep,
    age,
    gender, 
    trainingLevel,
    test_time,
    np.mean(recall_scores),
    np.mean(recog_scores)
  ])

  
  recall_scores = []
  recog_scores = []
  # 2nd post-test
  for y in ['A', 'B']:
    for x in row['second_posttest'][y]:
      if y == 'A':
        score = x['automated_recall_score']
        recall_scores.append(score)
      else:
        QID = x['QID']
        score = 1 if x['response'] == correct_answers[QID] else 0
        recog_scores.append(score)

  test_time = 7
  mat.append([
    meanSleep,
    varSleep,
    minSleep,
    maxSleep,
    lastSleep,
    age,
    gender, 
    trainingLevel,
    test_time,
    np.mean(recall_scores),
    np.mean(recog_scores)
  ])


  # 3rd post-test
  recall_scores = []
  recog_scores = []
  for y in ['A', 'B']:
    for _, x in row[f'posttest{y}'].iterrows():
      if y == 'A':
        score = x['automated_recall_score']
        recall_scores.append(score)
      else:
        QID = x['QID']
        score = 1 if x['response'] == correct_answers[QID] else 0
        recog_scores.append(score)

  test_time = 60
  mat.append([
    meanSleep,
    varSleep,
    minSleep,
    maxSleep,
    lastSleep,
    age,
    gender, 
    trainingLevel,
    test_time,
    np.mean(recall_scores),
    np.mean(recog_scores)
  ])


df = pd.DataFrame(mat, columns=['meanSleep', "varSleep", "minSleep", "maxSleep", "lastSleep",
                                "age", "gender", "trainingLevel", "test_time", "recallScore", "recogScore"])
df = pd.get_dummies(df, drop_first=True)

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

 
print("RECALL")
lm = ols('recallScore ~ meanSleep + varSleep + minSleep + maxSleep + lastSleep + age + gender_Male + trainingLevel + test_time', df).fit()
print(lm.summary())
anova_table = anova_lm(lm)
print('')
print(anova_table['sum_sq'] / anova_table['sum_sq'].sum())


In [ ]:
print("RECOGNITION")
lm = ols('recogScore ~ meanSleep + varSleep + minSleep + maxSleep + lastSleep + age + gender_Male + trainingLevel + test_time', df).fit()
print(lm.summary())
anova_table = anova_lm(lm)
print(anova_table['sum_sq'] / anova_table['sum_sq'].sum())


In [ ]:
# filter df to test_time = 1
df1 = df[df['test_time'] == 1]

# plot multiple scatterplots in one fig
fig, axs = plt.subplots(1, 5, figsize=(25, 4))

import statsmodels.api as sm

for i, var in enumerate(["meanSleep", "varSleep", "minSleep", "maxSleep", "lastSleep"]):
  axs[i].scatter(df1[var], df1['recallScore'])
  axs[i].set_title(var)

  # linear regression between maxSleep and score 
  x = df1[[var]]
  y = df1['recallScore']
  
  x = sm.add_constant(x) # adding a constant
  mod = sm.OLS(y, x)
  fii = mod.fit()

  # print R^2, coeff, and p-value from fii
  print(f"{var} | R^2: {fii.rsquared}, Coeff: {fii.params[1]}, p: {fii.pvalues[1]}")

  # plot line of best fit on meansleep plot
  x = np.linspace(np.min(df1[var]), np.max(df1[var]), 100)
  y = fii.params[0] + fii.params[1] * x
  axs[i].plot(x, y, color='red')

In [ ]:
# filter df to test_time = 1
df1 = df[df['test_time'] == 1]
df1

# plot multiple scatterplots in one fig
fig, axs = plt.subplots(1, 5, figsize=(25, 4))

for i, var in enumerate(["meanSleep", "varSleep", "minSleep", "maxSleep", "lastSleep"]):
  axs[i].scatter(df1[['trainingLevel']], df1[[var]])
  axs[i].set_title(var)

  # linear regression between maxSleep and score 
  x = df1[['trainingLevel']]
  y = df1[[var]]

  x = sm.add_constant(x) # adding a constant
  mod = sm.OLS(y, x)
  fii = mod.fit()

  # print R^2, coeff, and p-value from fii
  print(f"{var} | R^2: {fii.rsquared}, Coeff: {fii.params[1]}, p: {fii.pvalues[1]}")

  # plot line of best fit on meansleep plot
  x = np.linspace(np.min(df1['trainingLevel']), np.max(df1['trainingLevel']), 100)
  y = fii.params[0] + fii.params[1] * x
  axs[i].plot(x, y, color='red')

## Survey Results

In [ ]:
plt.rcParams.update({'font.size': 20})

# create pandas df from CSV file
df = pd.read_csv('/home/arpan/SAIL/paper/SAIL-Post-Survey.csv')
surveyQuestions = df.iloc[0]
surveyQuestions = surveyQuestions.to_dict()

# drop the 1st row from df
df = df.iloc[1:]

df = df[~df["2"].isin(bad_emails)]
emails = df["2"].unique()
print(emails, "num responses:", len(emails))

arr = [str(x) for x in range(12, 30)]
arr.append("35")
arr.append("3")
df[arr] = df[arr].apply(pd.to_numeric)
surveyQuestions

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4), dpi=300)
bar_width = 0.2
ax.set_ylim(-1, 13)
# ax.bar(np.arange(4) - bar_width,
#        [np.mean(df["12"]), np.mean(df["13"]), np.mean(df["14"]), np.mean(df["15"])], width=bar_width, label='Oral', color='#1b9e77', align="center")
# ax.bar(np.arange(4),
#        [np.mean(df["16"]), np.mean(df["17"]), np.mean(df["18"]), np.mean(df["19"])], width=bar_width, label='Typed', color='#0295D9', align="center")
# ax.bar(np.arange(4) + bar_width,
#        [np.mean(df["20"]), np.mean(df["21"]), np.mean(df["22"]), np.mean(df["23"])], width=bar_width, label='MC', color='#7570b3', align="center")

# plot box-and-whisker plot for each question
c='#1b9e77'
bp1 = ax.boxplot([df["12"], df["13"], df["14"], df["15"]], positions=np.arange(4) - bar_width - 0.05, widths=bar_width, showfliers=False,
                 patch_artist=True, boxprops=dict(fc=c, color='black'), 
                 medianprops=dict(color='black'), flierprops=dict(color=c, markeredgecolor=c))

c='#0295D9'
bp2 = ax.boxplot([df["16"], df["17"], df["18"], df["19"]], positions=np.arange(4), widths=bar_width, showfliers=False,
                 patch_artist=True, boxprops=dict(fc=c, color='black'), 
                 medianprops=dict(color='black'), flierprops=dict(color=c, markeredgecolor=c))

c='#7570b3'
bp3 = ax.boxplot([df["20"], df["21"], df["22"], df["23"]], positions=np.arange(4) + bar_width + 0.05, widths=bar_width, showfliers=False,
                 patch_artist=True, boxprops=dict(fc=c, color='black'), 
                 medianprops=dict(color='black'), flierprops=dict(color=c, markeredgecolor=c))


ax.set_xticks(np.arange(4),
              ['Easy', 'Fun', "Effective", 'Efficient'], rotation=0, ha='center')


ax.set_ylabel('Mean rating', labelpad=10)
# ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=False, ncol=5)
ax.legend([bp1["boxes"][0], bp2["boxes"][0], bp3["boxes"][0]], ['Oral', 'Typed', 'MC'], 
          loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=False, ncol=5)

# set y ticks
ax.set_yticks(np.arange(0, 11, 2.5))

# make font sizes way bigger
plt.rcParams.update({'font.size': 20})
plt.show()

# save fig 600 dpi white bg
fig.savefig('ratings.png', dpi=600, bbox_inches='tight', transparent=False, pad_inches=0.1, facecolor='white')

In [ ]:
# ANOVA to compare modalities for each of these survey questions (Likert scale comparison)

for i, var in enumerate(["Easy", "Fun", "Effective", "Efficient"]):
  df1 = pd.DataFrame()
  df1["rating"] = df[f"{12+i}"]
  df1["modality"] = "oral"

  df2 = pd.DataFrame()
  df2["rating"] = df[f"{16+i}"]
  df2["modality"] = "typed"

  df3 = pd.DataFrame()
  df3["rating"] = df[f"{20+i}"]
  df3["modality"] = "mc"

  df1 = pd.concat([df1, df2, df3])

  # run a friedman to compare modalities
  anova = friedmanchisquare(df1[df1['modality'] == "oral"]['rating'].to_numpy(), 
                            df1[df1['modality'] == "typed"]['rating'].to_numpy(), 
                            df1[df1['modality'] == "mc"]['rating'].to_numpy())
  # print p-val
  print(var, anova.pvalue)

  # for modality in ["oral", "typed", "mc"]:
  #   print(scipy.stats.shapiro(df1[df1['modality'] == modality]['rating'].to_numpy()))

  # # # run turkey hsd to compare modalities
  # # from statsmodels.stats.multicomp import pairwise_tukeyhsd
  # # m_comp = pairwise_tukeyhsd(endog=df1['rating'], groups=df1['modality'], alpha=0.05)

  # # print(m_comp)

  # run a nonparametric repeated measures test to compare modalities
  stat, p = wilcoxon(df1[df1['modality'] == "oral"]['rating'].to_numpy(), 
                     df1[df1['modality'] == "typed"]['rating'].to_numpy(),
                     correction=True)
  print(f"Oral != Typed: {p}, reject: {True if p < (0.05/3) else False}")

  stat, p = wilcoxon(df1[df1['modality'] == "oral"]['rating'].to_numpy(), 
                     df1[df1['modality'] == "mc"]['rating'].to_numpy(),
                     correction=True)
  print(f"Oral != MC: {p}, reject: {True if p < (0.05/3) else False}")

  stat, p = wilcoxon(df1[df1['modality'] == "typed"]['rating'].to_numpy(), 
                     df1[df1['modality'] == "mc"]['rating'].to_numpy(),
                     correction=True)
  print(f"Typed != MC: {p}, reject: {True if p < (0.05/3) else False}")

  print('')

In [ ]:
casual = []
clinical = []
exam = []

for index, i in enumerate([7, 8, 9]):
  qNum = str(i)

  # in df[qNum], replace all instances of "Multiple choice" with "MC"
  df[qNum] = df[qNum].replace("Multiple choice", "MC")
  df[qNum] = df[qNum].replace("Multiple Choice", "MC")
  df[qNum] = df[qNum].replace("SAIL study app with voice", "Oral")
  df[qNum] = df[qNum].replace("Written response", "Typed")
  df[qNum] = df[qNum].replace("Written/typed response", "Typed")

  counts = []
  labels = ["Oral", "Typed", "MC"]
  for modality in labels:
    if modality in df[qNum].value_counts():
      counts.append(df[qNum].value_counts()[modality])
    else:
      counts.append(0)
  
  # change counts to a percentage of the sum of counts
  original_counts = counts
  counts = [x / sum(counts) for x in counts]
  if i == 7:
    casual = counts
    casual_original = original_counts
  elif i == 8:
    clinical = counts
    clinical_original = original_counts
  elif i == 9:
    exam = counts
    exam_original = original_counts

print(casual, clinical, exam)

fig, ax = plt.subplots(1, 1, figsize=(10, 4), dpi=600)
bar_width = 0.2
ax.set_ylim(0, 1)

ax.bar(np.arange(3) - bar_width,
       [x[0] for x in [casual, clinical, exam]], width=bar_width, label='Oral', color='#1b9e77', align="center", ec='black')
ax.bar(np.arange(3),
       [x[1] for x in [casual, clinical, exam]], width=bar_width, label='Typed', color='#0295D9', align="center", ec='black')
ax.bar(np.arange(3) + bar_width,
       [x[2] for x in [casual, clinical, exam]], width=bar_width, label='MC', color='#7570b3', align="center", ec='black')

ax.set_xticks(np.arange(3),
              ['Casual Learning', 'Clinical Prep', "Exam Prep"], rotation=0, ha='center')


ax.set_ylabel('Proportion', labelpad=10)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=False, ncol=5)

plt.show()

fig.savefig('proportions_per_setting.png', dpi=600, bbox_inches='tight', transparent=False, pad_inches=0.1, facecolor='white')

In [ ]:
# Compare proportions

for var in ["Casual", "Clinical Prep", "Exam Prep"]:
  
  metric = None
  if var == "Casual":
    metric = casual_original
  elif var == "Clinical Prep":
    metric = clinical_original
  elif var == "Exam Prep":
    metric = exam_original
  
  # chi square
  f_obs = metric # [oral, typed, mc]
  chi2, p = scipy.stats.chisquare(f_obs)
  print(f"{var} --> chi-square p: {p}")

  if var == 'Clinical Prep':
    print('')
    continue

  # posthoc
  for i, modality in enumerate(["Oral", "Typed", "MC"]):
    # drop i from f_exp
    f_obs = [x for j, x in enumerate(metric) if j != i]
    chi2, p = scipy.stats.chisquare(f_obs)

    modalities = [x for j,x in enumerate(["Oral", "Typed", "MC"]) if j != i]
    print(f"{modalities} --> chi-square p: {p}, reject = {True if p < (0.05/3) else False}")
  

  print('')

In [ ]:
#Which learning modality did you like/enjoy the most?
df['6'].value_counts(normalize=True)

In [ ]:
#Would you use SAIL in addition to your current study methods?
df['32'].value_counts(normalize=True)

In [ ]:
# As a result of using SAIL written and voice modes, do you feel you gained knowledge that can help you score higher 
# on an orthopedics in-training exam or written boards exam?
df['11'].value_counts(normalize=True)

## Automatic Free Response Grading

In [ ]:
import os
import sys

sys.path.append(os.path.abspath('../backend/scoring'))

In [ ]:
import score

scorer = score.new_scorer(root='../backend/scoring', verbose=False)

In [ ]:
for i in range(len(finished_post_data)):
    for posttest_iteration in ['first_posttest', 'second_posttest', "posttestA"]:
        if posttest_iteration == "posttestA":
            posttestA = finished_post_data.iloc[i][posttest_iteration]
        else:
            posttestA = finished_post_data.iloc[i][posttest_iteration]['A']
        
        # if posttest_iteration == "posttestA":
        #     print(posttestA.iloc[0]['start'])
        for j in range(len(posttestA)):
            if posttest_iteration == "posttestA":
                # print(posttestA.iloc[j])
                # print(finished_post_data.iloc[i]['email'], posttest_iteration, posttestA.iloc[j]['QID'], posttestA.iloc[j]["response"])
                posttestA.at[j, "automated_recall_score"] = scorer.score(posttestA.iloc[j]['QID'], posttestA.iloc[j]["response"])
                pass
            else: 
                # print(finished_post_data.iloc[i]['email'], posttest_iteration, posttestA[j]['QID'], posttestA[j]["response"])
                posttestA[j]["automated_recall_score"] = scorer.score(posttestA[j]['QID'], posttestA[j]["response"])

In [ ]:
pd.DataFrame(finished_post_data["posttestA"][2])

In [ ]:
pd.DataFrame(finished_post_data["first_posttest"][0]["A"])

In [ ]:
for posttest_iteration in ['posttestA', 'first_posttest', 'second_posttest']:
    print(f"Scoring {posttest_iteration}")
    for email, posttest in tqdm.tqdm(list(zip(finished_post_data['email'], finished_post_data[posttest_iteration]))):
        with open(f'../backend/users/{email}.yaml') as f:
            user_data = yaml.safe_load(f)
        
        if posttest_iteration != "posttestA": 
            for q, posttest_q in zip(user_data[posttest_iteration]["A"], posttest["A"]):
                q['automated_recall_score'] = posttest_q["automated_recall_score"]
        else:
            posttest_array = [posttest.iloc[i].to_dict() for i in range(posttest.shape[0])]
            for q, posttest_q in zip(user_data[posttest_iteration], posttest_array):
                q['automated_recall_score'] = int(posttest_q["automated_recall_score"])
        
        with open(f'../backend/users/{email}.yaml', 'w') as f:
            yaml.dump(user_data, f)

## Sleep Data

In [ ]:
sleep=finished_post_data['sleepData'].to_numpy()
sleep=np.concatenate(sleep)
plt.hist(sleep)
plt.show()

print("Sleep")
print(f"Min: {np.min(sleep)}")
print(f"Median: {np.median(sleep)}")
print(f"Max: {np.max(sleep)}")
print(f"Mean: {np.mean(sleep)}")
print(f"SEM: {scipy.stats.sem(sleep)}")


# average sleep hrs might be an important factor
# but also variability in sleep for a person might be important
# or min